--- Day 10: Pipe Maze ---
You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This island is surprisingly cold and there definitely aren't any thermals to glide on, so you leave your hang glider behind.

You wander around for a while, but you don't find any people or animals. However, you do occasionally find signposts labeled "Hot Springs" pointing in a seemingly consistent direction; maybe you can find someone at the hot springs and ask them where the desert-machine parts are made.

The landscape here is alien; even the flowers and trees are made of metal. As you stop to admire some metal grass, you notice something metallic scurry away in your peripheral vision and jump into a big pipe! It didn't look like any animal you've ever seen; if you want a better look, you'll need to get ahead of it.

Scanning the area, you discover that the entire field you're standing on is densely packed with pipes; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).

The pipes are arranged in a two-dimensional grid of tiles:

| is a vertical pipe connecting north and south.
- is a horizontal pipe connecting east and west.
L is a 90-degree bend connecting north and east.
J is a 90-degree bend connecting north and west.
7 is a 90-degree bend connecting south and west.
F is a 90-degree bend connecting south and east.
. is ground; there is no pipe in this tile.
S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.
Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is one large, continuous loop.

For example, here is a square loop of pipe:

.....
.F-7.
.|.|.
.L-J.
.....
If the animal had entered this loop in the northwest corner, the sketch would instead look like this:

.....
.S-7.
.|.|.
.L-J.
.....
In the above diagram, the S tile is still a 90-degree F bend: you can tell because of how the adjacent pipes connect to it.

Unfortunately, there are also many pipes that aren't connected to the loop! This sketch shows the same loop as above:

-L|F7
7S-7|
L|7||
-L-J|
L|-JF
In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to S, pipes those pipes connect to, pipes those pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including S, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).

Here is a sketch that contains a slightly more complex main loop:

..F7.
.FJ|.
SJ.L7
|F--J
LJ...
Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:

7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ
If you want to get out ahead of the animal, you should find the tile in the loop that is farthest from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps along the loop to reach from the starting point - regardless of which way around the loop the animal went.

In the first example with the square loop:

.....
.S-7.
.|.|.
.L-J.
.....
You can count the distance each tile in the loop is from the starting point like this:

.....
.012.
.1.3.
.234.
.....
In this example, the farthest point from the start is 4 steps away.

Here's the more complex loop again:

..F7.
.FJ|.
SJ.L7
|F--J
LJ...
Here are the distances for each tile on that loop:

..45.
.236.
01.78
14567
23...
Find the single giant loop starting at S. How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?

In [1]:
from dataclasses import dataclass
from __future__ import annotations


import numpy as np


@dataclass
class PipeMaze:
    maze_array: np.ndarray
    start_index: tuple[int, int]


    @classmethod
    def from_str(cls, input_str: str) -> PipeMaze:
        maze_list = input_str.strip().split("\n")
        n_lines = len(maze_list)
        len_line = len(maze_list[0])
        maze_array = np.empty((n_lines, len_line), dtype=np.dtype("U"))
        for i, line_str in enumerate(maze_list):
            for j, value in enumerate(line_str):
                maze_array[i, j] = value
                if value == "S":
                    start_index = (i, j)
        return cls(
            maze_array=maze_array,
            start_index=start_index,
            )


    def find_connected_nodes(self, node_index: tuple(int, int)) -> list[tuple[int, int]]:
        connected_nodes = []
        # check above
        if node_index[0] > 0:
            above_value = self.maze_array[node_index[0] - 1, node_index[1]]
            if above_value == "|":
                connected_nodes.append((node_index[0] - 2, node_index[1]))
            elif above_value == "F":
                connected_nodes.append((node_index[0] - 1, node_index[1] + 1))
            elif above_value == "7":
                connected_nodes.append((node_index[0] - 1, node_index[1] - 1))

        # check below
        if node_index[0] < self.maze_array.shape[0] - 1:
            below_value = self.maze_array[node_index[0] + 1, node_index[1]]
            if below_value == "|":
                connected_nodes.append((node_index[0] + 2, node_index[1]))
            elif below_value == "L":
                connected_nodes.append((node_index[0] + 1, node_index[1] + 1))
            elif below_value == "J":
                connected_nodes.append((node_index[0] + 1, node_index[1] - 1))
        
        # check left
        if node_index[1] > 0:
            left_value = self.maze_array[node_index[0], node_index[1] - 1]
            if left_value == "-":
                connected_nodes.append((node_index[0], node_index[1] - 2))
            elif left_value == "L":
                connected_nodes.append((node_index[0] - 1 , node_index[1] - 1))
            elif left_value == "F":
                connected_nodes.append((node_index[0] + 1, node_index[1] - 1))

        # check right
        if node_index[1] < self.maze_array.shape[1] - 1:
            right_value = self.maze_array[node_index[0], node_index[1] + 1]
            if right_value == "-":
                connected_nodes.append((node_index[0], node_index[1] + 2))
            elif right_value == "J":
                connected_nodes.append((node_index[0] - 1, node_index[1] + 1))
            elif right_value == "7":
                connected_nodes.append((node_index[0] + 1, node_index[1] + 1))

        connected_nodes = list(filter(lambda x: -1 < x[0] < self.maze_array.shape[0] - 1 and -1 < x[1] < self.maze_array.shape[1] - 1, connected_nodes))

        return connected_nodes
    
    @property
    def node_map(self) -> dict[tuple[int, int], list[tuple[int, int]]]:
        node_map = {}
        for i in range(self.maze_array.shape[0]):
            for j in range(self.maze_array.shape[1]):
                node_map[(i, j)] = self.find_connected_nodes((i, j))
        return node_map



    def find_max_steps(self) -> int:
        node_to_steps_map = {self.start_index: 0}
        current_positions = [self.start_index]
        n_steps = 0
        any_nodes = True
        step_limit = 10000
        while any_nodes:
            n_steps += 2
            next_positions = []
            for curr_idx in current_positions:
                for node in self.node_map[curr_idx]:
                    if node not in node_to_steps_map:
                        next_positions.append(node)
                        node_to_steps_map[node] = n_steps

            if n_steps % 100 == 0:
                print(n_steps)
                print(node_to_steps_map)
            current_positions = next_positions
            any_nodes = bool(current_positions)
            if n_steps > step_limit:
                return Exception(f"Exceed step limit of {step_limit}")
        return node_to_steps_map


In [2]:
test_input = """
.....
.S-7.
.|.|.
.L-J.
.....
.....
"""

In [3]:
test_maze = PipeMaze.from_str(test_input)

In [4]:
test_maze.maze_array

array([['.', '.', '.', '.', '.'],
       ['.', 'S', '-', '7', '.'],
       ['.', '|', '.', '|', '.'],
       ['.', 'L', '-', 'J', '.'],
       ['.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.']], dtype='<U1')

In [5]:
test_maze.find_connected_nodes((1, 1))

[(3, 1), (1, 3)]

In [6]:
test_maze.find_connected_nodes((3, 1))

[(1, 1), (3, 3)]

In [7]:
test_maze.find_max_steps()

{(1, 1): 0, (3, 1): 2, (1, 3): 2, (3, 3): 4}

In [8]:
with open("../inputs/day10_input.txt") as f:
    input_str = f.read()


maze = PipeMaze.from_str(input_str)

In [9]:
maze.find_max_steps()

100
{(22, 91): 0, (21, 90): 2, (22, 89): 2, (20, 89): 4, (23, 90): 4, (19, 90): 6, (24, 91): 6, (18, 89): 8, (24, 89): 8, (18, 91): 10, (25, 90): 10, (18, 93): 12, (26, 91): 12, (19, 92): 14, (27, 92): 14, (20, 91): 16, (25, 92): 16, (28, 91): 16, (21, 92): 18, (26, 93): 18, (30, 91): 18, (29, 90): 18, (23, 92): 20, (28, 93): 20, (30, 89): 20, (29, 88): 20, (24, 93): 22, (29, 92): 22, (31, 88): 22, (28, 87): 22, (22, 93): 24, (30, 93): 24, (32, 89): 24, (28, 89): 24, (20, 93): 26, (32, 93): 26, (32, 87): 26, (27, 90): 26, (21, 94): 28, (33, 94): 28, (34, 87): 28, (26, 89): 28, (23, 94): 30, (34, 95): 30, (36, 87): 30, (27, 88): 30, (24, 95): 32, (35, 94): 32, (35, 88): 32, (26, 87): 32, (25, 94): 34, (34, 93): 34, (33, 88): 34, (24, 87): 34, (27, 94): 36, (36, 93): 36, (34, 89): 36, (23, 88): 36, (29, 94): 38, (36, 95): 38, (35, 90): 38, (22, 87): 38, (31, 94): 40, (37, 94): 40, (36, 89): 40, (21, 88): 40, (32, 95): 42, (38, 93): 42, (37, 90): 42, (20, 87): 42, (30, 95): 44, (39, 94): 